In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from gmplot import gmplot
from sklearn import metrics
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')
# os.chdir("../")
# os.listdir()

In [2]:
city_name = 'delhi-1'

In [20]:
file_name = 'data/' + city_name + '-reduced_amenity.csv'
data = pd.read_csv(file_name)
data_amenities = data.copy()
data = data[['lat', 'lon']]
print(data.shape)
data.head()

(3499, 2)


,lat,lon
0,28.533492,77.151895
1,28.533647,77.152001
2,28.489238,77.090988
3,28.450926,77.087609
4,28.440015,77.102391


In [21]:
coords = data.values

# DBSCAN

In [22]:
kms_per_radian = 6371.0088
epsilon = 1 / kms_per_radian

dbscan = DBSCAN(eps = epsilon ,
                min_samples = 2.2 ,
                algorithm = 'ball_tree' ,
                metric = 'haversine')

model = dbscan.fit(np.radians(coords))


In [23]:
cluster_labels = dbscan.labels_
num_clusters = len(set(cluster_labels))
num_clusters

44

In [24]:
clusters = pd.Series([ coords [ cluster_labels == n ] for n in range (num_clusters) ])
clusters[0], len(clusters[0])

(array([[28.533492 , 77.1518947],
        [28.5336472, 77.152001 ],
        [28.5084497, 77.166546 ],
        ...,
        [28.509824 , 77.201705 ],
        [28.5100591, 77.201592 ],
        [28.6406954, 77.2113562]]), 2536)

In [25]:
# clusters = clusters[1:]

In [26]:
# clusters[101]

In [27]:
core_samples = np.zeros_like(cluster_labels, dtype='bool')
core_samples[dbscan.core_sample_indices_] = True
core_samples

array([ True,  True,  True, ...,  True,  True,  True])

In [28]:
len(core_samples), len(data)

(3499, 3499)

In [29]:
np.unique(core_samples, return_counts=True)

(array([False,  True]), array([ 102, 3397]))

In [31]:
s = pd.Series(core_samples, name='bools')

In [32]:
data[s.values].shape, data.shape

((3397, 2), (3499, 2))

In [40]:
df_amenities = data_amenities[s.values]
print(df_amenities.shape)
df_amenities.head()

(3397, 5)


,node_id,lat,lon,name,amenity
0,248852574,28.533492,77.151895,Cafe Coffee Day,cafe
1,248852583,28.533647,77.152001,"ICICI, SBI, Citibank,",bank
2,249054695,28.489238,77.090988,NaN,fuel
3,249132373,28.450926,77.087609,Paras Hospitals,hospital
4,249132375,28.440015,77.102391,HDFC Bank & ATM,bank


In [42]:
file_name = "data/" + city_name + '-dbscan.csv'
df_amenities.to_csv(file_name, index=False)

Aakash Jhawar https://aakashjhawar.github.io